In [1]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import time
import glob
class HandGesturePreprocessor:
    def __init__(self, target_size=(224, 224), normalization=True, augmentation_params=None, save_dir=None, use_edge_detection=False, enable_augmentation=False):
        """
        Initialize the preprocessor.
        :param target_size: Tuple, the target size of the images (width, height).
        :param normalization: Bool, whether to normalize the images to the range [0, 1].
        :param augmentation_params: Dict, parameters for data augmentation. If None, default params are used.
        :param save_dir: String, directory to save augmented images. If None, images are not saved.
        :param use_edge_detection: Bool, whether to apply edge detection in the preprocessing pipeline.
        :param enable_augmentation: Bool, whether to enable data augmentation.
        """
        self.target_size = target_size
        self.normalization = normalization
        self.save_dir = save_dir
        self.use_edge_detection = use_edge_detection
        self.enable_augmentation = enable_augmentation
        if augmentation_params is None:
            self.augmentation_params = {
                'rotation_range': 20,
                'width_shift_range': 0.2,
                'height_shift_range': 0.2,
                'shear_range': 0.2,
                'zoom_range': 0.2,
                'horizontal_flip': True,
                'fill_mode': 'nearest'
            }
        else:
            self.augmentation_params = augmentation_params
        self.data_gen = ImageDataGenerator(**self.augmentation_params)

    def preprocess(self, image):
        """
        Preprocess the image: resize, optionally apply edge detection, and normalize.
        :param image: Numpy array, the image to preprocess.
        :return: Preprocessed image with 4 dimensions, ready for augmentation.
        """
        # Convert to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Resize image
        image_resized = cv2.resize(image, self.target_size)

        # Optionally apply edge detection
        if self.use_edge_detection:
            gray = cv2.cvtColor(image_resized, cv2.COLOR_RGB2GRAY)
            edges = cv2.Canny(gray, 100, 200)
            image_processed = np.expand_dims(edges, axis=-1)  # Add channel dimension
        else:
            image_processed = image_resized

        # Normalize if required
        if self.normalization:
            image_processed = image_processed.astype('float32') / 255.0

        # Ensure image has 4 dimensions (1, height, width, channels)
        if image_processed.ndim == 3:
            image_processed = np.expand_dims(image_processed, axis=0)

        return image_processed

    def augment_and_save(self, image, augmentations=5):
        """
        Apply data augmentation and save the augmented images.
        :param image: 4D Numpy array, the preprocessed image for augmentation.
        :param augmentations: Int, number of augmented images to generate.
        """
        if self.enable_augmentation:
            for _ in range(augmentations):
                it = self.data_gen.flow(image, batch_size=1, save_to_dir=self.save_dir, save_prefix='aug', save_format='jpeg')
                next(it)
        else:
            print("Augmentation is disabled. Skipping augmentation step.")

    def augment_and_save_folder(self, folder_path, augmentations=5):
        """
        Apply data augmentation and save for all images in a folder.
        :param folder_path: String, the path to the folder containing images.
        :param augmentations: Int, number of augmented images to generate per original image.
        """
        # Ensure folder_path exists and is a directory
        if not os.path.exists(folder_path) or not os.path.isdir(folder_path):
            raise ValueError(f"Provided folder path '{folder_path}' is not valid.")

        # List all image files in the directory
        image_files = glob.glob(os.path.join(folder_path, '*'))

        # Iterate over each file
        for image_file in image_files:
            # Load the image
            image = cv2.imread(image_file)
            if image is not None:
                # Preprocess the image
                preprocessed_image = self.preprocess(image)

                # Augment the preprocessed image and save the augmented images to disk
                self.augment_and_save(preprocessed_image, augmentations=augmentations)
            else:
                print(f"Error loading image from {image_file}. Skipping this file.")


    def save_image(self, image, save_prefix='image', save_format='jpg'):
        """
        Save the preprocessed (or augmented) image to disk in the directory specified by save_dir.
        Generates a filename using a prefix and a timestamp.
        :param image: 4D Numpy array, the image to save.
        :param save_prefix: Prefix for generating the filename.
        :param save_format: Image format (e.g., 'jpg', 'png').
        """
        if self.save_dir is None:
            raise ValueError("Save directory not specified.")

        # Ensure the save directory exists
        if not os.path.exists(self.save_dir):
            os.makedirs(self.save_dir, exist_ok=True)

        # Generate a unique filename using the prefix and the current timestamp
        timestamp = int(time.time() * 1e6)  # Microsecond timestamp to ensure uniqueness
        filename = f"{save_prefix}_{timestamp}.{save_format}"
        file_path = os.path.join(self.save_dir, filename)

        # Remove batch dimension if present
        if image.shape[0] == 1:
            image = image.squeeze(0)

        # Normalize and convert color space if necessary
        if self.normalization:
            image = (image * 255).astype(np.uint8)
        if image.ndim == 2 or image.shape[-1] == 1:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

        # Save the image
        cv2.imwrite(file_path, image)
        print(f"Image saved to {file_path}")




Using Augmentation


In [2]:
import cv2

# Path to your training image
image_path = '/content/drive/MyDrive/Colab Notebooks/Group_Project/Input_Images/test/frame_00_06_0019.png'

# Initialize the preprocessor with augmentation enabled for training
preprocessor = HandGesturePreprocessor(
    target_size=(224, 224),
    normalization=True,
    use_edge_detection=True,  # Edge detection could be enabled based on your needs
    enable_augmentation=True,
    save_dir='/content/drive/MyDrive/Colab Notebooks/Group_Project/Output_Images/Augmented_Images/test'  # Ensure this directory exists
)

# Load the training image
image = cv2.imread(image_path)
if image is not None:
    # Preprocess the image
    preprocessed_image = preprocessor.preprocess(image)

    # Augment the preprocessed image and save the augmented images to disk
    preprocessor.augment_and_save(preprocessed_image, augmentations=10)
else:
    print(f"Error loading image from {image_path}. Please check the path.")


Without Augmentation

In [ ]:
import cv2

# Path to your validation/testing image
image_path = '/content/drive/MyDrive/Colab Notebooks/Group_Project/Input_Images/elena-rabkina-QH8aF3B0gYQ-unsplash.jpg'

# Initialize the preprocessor without augmentation for validation/testing
preprocessor_wo = HandGesturePreprocessor(
    target_size=(224, 224),
    normalization=True,
    use_edge_detection=False,  # This can be true if you want to use edge detection
    enable_augmentation=False,  # Important for validation/testing
    save_dir='/content/drive/MyDrive/Colab Notebooks/Group_Project/Output_Images/Without_Augmentation'
)

# Load the validation/testing image
image = cv2.imread(image_path)
if image is not None:
    # Preprocess the image without augmentation
    preprocessed_image = preprocessor_wo.preprocess(image)

    # Optionally save the preprocessed image for inspection
    #save_path = '/content/drive/MyDrive/Colab Notebooks/Group_Project/Output_Images/Without_Augmentation.jpg'
    preprocessor_wo.save_image(preprocessed_image,)
else:
    print(f"Error loading image from {image_path}. Please check the path.")


Image saved to /content/drive/MyDrive/Colab Notebooks/Group_Project/Output_Images/Without_Augmentation/image_1708066690423163.jpg


With Augmentation for entire file

In [ ]:
import cv2
import os

image_folder_path = '/content/drive/MyDrive/Colab Notebooks/Group_Project/Input_Images/Thumbs Up/Thumbs up'
preprocessor = HandGesturePreprocessor(
    target_size=(224, 224),
    normalization=True,
    use_edge_detection=True,
    enable_augmentation=True,
    save_dir='/content/drive/MyDrive/Colab Notebooks/Group_Project/Output_Images/Augmented_Images/THUMBS_UP'
)

# Apply augmentation to all images in the specified folder
preprocessor.augment_and_save_folder(image_folder_path, augmentations=10)
